In [ ]:
import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import h5py
from zipfile import ZipFile
import gdal
from osgeo import gdal
import os
from ftplib import FTP
from osgeo import ogr, gdal

print('Libraries imported')

Libraries imported


In [ ]:
username = '' #Input username here
password = '' #Input username here


In [ ]:
ftp = FTP()
ftp.connect('ftp.mosdac.gov.in', 21)
ftp.login(username,password)
ftp.retrlines("LIST")
print('Go to next cell')

drwxr-xr-x    3 0        0            4096 Mar 05 08:24 Order
Go to next cell


In [ ]:
ftp.dir()

drwxr-xr-x    3 0        0            4096 Mar 05 08:24 Order


In [ ]:
ftp.cwd('Order')
ftp.pwd()
ftp.dir()

drwxr-xr-x    2 0        0            8192 Mar 05 08:27 Mar20_059265


In [ ]:
ftp.cwd('Mar20_059265')
ftp.pwd()
ftp.dir()

-rw-r--r--    1 0        0          794866 Mar 05 08:24 3DIMG_07NOV2019_0000_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          797436 Mar 05 08:24 3DIMG_07NOV2019_0030_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          795533 Mar 05 08:24 3DIMG_07NOV2019_0100_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          788948 Mar 05 08:24 3DIMG_07NOV2019_0130_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          790983 Mar 05 08:24 3DIMG_07NOV2019_0200_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          785626 Mar 05 08:24 3DIMG_07NOV2019_0230_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          792345 Mar 05 08:24 3DIMG_07NOV2019_0300_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          779977 Mar 05 08:24 3DIMG_07NOV2019_0330_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          777272 Mar 05 08:24 3DIMG_07NOV2019_0400_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          776842 Mar 05 08:24 3DIMG_07NOV2019_0430_L2B_CMK_CMK.tif
-rw-r--r--    1 0        0          779474 Mar 05 08:24 3DIMG_07NOV201

In [ ]:
"""Function to crop the input tif image with reference input shape file and
resample to the given resolution
INPUTS 
InputImage - tif image as a raster
Shapefile - Reference shapefile, the supporting files should also be present in the directory
PixelRes - Resolution of pixel, with default values as 0.03353227354096834784, that 
results in an image with ncol,nrow as 984,1074

Outputs - An cropped image is saved in the library with a word '_cropped' appeneded to the original name 
"""

def crop_resample(InputImage,Shapefile,PixelRes = 0.03353227354096834784):
    RasterFormat = 'GTiff'
    VectorFormat = 'ESRI Shapefile'
    # Open datasets
    Raster = gdal.Open(InputImage, gdal.GA_ReadOnly)
    #print(type(Raster))
    Projection = Raster.GetProjectionRef()

    VectorDriver = ogr.GetDriverByName(VectorFormat)
    VectorDataset = VectorDriver.Open(Shapefile, 0) # 0=Read-only, 1=Read-Write
    #print(type(VectorDataset))

    layer = VectorDataset.GetLayer()

    feature =layer.GetFeature(0)
    # Iterate through the shapefile features
    #print(type(layer))
    geom = feature.GetGeometryRef() 
    minX, maxX, minY, maxY = geom.GetEnvelope() # Get bounding box of the shapefile feature

    # Create raster
    OutTileName = str(InputImage)+'_cropped.tif'
    OutTile = gdal.Warp(OutTileName, Raster, format=RasterFormat, outputBounds=[minX, minY, maxX, maxY], xRes=PixelRes, yRes=PixelRes, dstSRS=Projection, resampleAlg=gdal.GRA_Bilinear, options=['COMPRESS=DEFLATE'])
    OutTile = None # Close dataset

    # Close datasets
    Raster = None
    VectorDataset.Destroy()
    print(OutTileName+"  Created")



In [ ]:
ftp.quit()

AttributeError: ignored

https://gis.stackexchange.com/questions/203664/clipping-tiff-raster-image-using-bounding-box-with-gdal-in-python
